In [24]:
import os
from tensorflow.keras.layers import Flatten,Dense

In [31]:
from glob import glob
from tensorflow.keras.models import Model

In [2]:
from tensorflow.keras.applications.resnet import ResNet50

In [3]:
ORIG_INPUT_DATASET = "cell_images"

In [4]:
BASE_PATH = os.getcwd()

In [5]:
TRAIN_PATH = os.path.sep.join([BASE_PATH, "training"])
VAL_PATH = os.path.sep.join([BASE_PATH, "validation"])
TEST_PATH = os.path.sep.join([BASE_PATH, "testing"])

In [6]:
Train_splt = 0.8
val_split = 0.1

In [7]:
from imutils import paths
import random
import shutil

In [8]:
imagePaths = list(paths.list_images(ORIG_INPUT_DATASET))
random.seed(42)
random.shuffle(imagePaths)

In [9]:
i = int(len(imagePaths)*Train_splt)
trainPaths = imagePaths[:i]
testPaths = imagePaths[i:]

i = int(len(imagePaths)*val_split)
valPaths = trainPaths[:i]
trainPaths =trainPaths[i:] 

In [10]:
datasets = [("training",trainPaths,TRAIN_PATH),
           ("validation",valPaths,VAL_PATH),
           ("testing",testPaths,TEST_PATH)]

In [11]:
# loop over the datasets
for (dType, imagePaths, baseOutput) in datasets:
	# show which data split we are creating
	print("[INFO] building '{}' split".format(dType))
	# if the output base output directory does not exist, create it
	if not os.path.exists(baseOutput):
		print("[INFO] 'creating {}' directory".format(baseOutput))
		os.makedirs(baseOutput)
	# loop over the input image paths
	for inputPath in imagePaths:
		# extract the filename of the input image along with its
		# corresponding class label
		filename = inputPath.split(os.path.sep)[-1]
		label = inputPath.split(os.path.sep)[-2]
		# build the path to the label directory
		labelPath = os.path.sep.join([baseOutput, label])
		# if the label output directory does not exist, create it
		if not os.path.exists(labelPath):
			print("[INFO] 'creating {}' directory".format(labelPath))
			os.makedirs(labelPath)
		# construct the path to the destination image and then copy
		# the image itself
		p = os.path.sep.join([labelPath, filename])
		shutil.copy2(inputPath, p)

[INFO] building 'training' split
[INFO] building 'validation' split
[INFO] building 'testing' split


In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers import SGD
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np

C:\Users\Sujay J\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Sujay J\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Sujay J\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Sujay J\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [13]:
plott = os.getcwd()

In [14]:
NUM_EPOCHS = 1
INIT_LR = 1e-1
BS = 32


In [15]:
def poly_decay(epoch):
    maxEpochs = NUM_EPOCHS
    baseLR = INIT_LR
    power = 1.0
    alpha= baseLR *(1-(epoch/float(maxEpochs)))**power
    return alpha

In [16]:
totalTrain = len(list(paths.list_images(TRAIN_PATH)))
totalVal = len(list(paths.list_images(VAL_PATH)))
totalTest = len(list(paths.list_images(TEST_PATH)))

In [17]:
trainAug = ImageDataGenerator(
    rescale=1 / 255.0,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.05,
    height_shift_range=0.05,
    shear_range=0.05,
    horizontal_flip=True,
    fill_mode="nearest")

In [18]:
valAug = ImageDataGenerator(rescale=1 / 255.0)

In [19]:
trainGen = trainAug.flow_from_directory(TRAIN_PATH,class_mode="categorical",
                                       target_size=(64,64),color_mode="rgb",
                                       shuffle=False,batch_size=BS)

Found 19291 images belonging to 2 classes.


In [20]:
valGen = valAug.flow_from_directory(VAL_PATH,class_mode="categorical",
                                   target_size=(64,64),color_mode="rgb",
                                   shuffle=False,batch_size=BS)

Found 2755 images belonging to 2 classes.


In [21]:
testGen = valAug.flow_from_directory(TEST_PATH,class_mode="categorical",
                                   target_size=(64,64),color_mode="rgb",
                                   shuffle=False,batch_size=BS)

Found 5512 images belonging to 2 classes.


In [22]:
IMAGE_Size = [64,64,3]

In [23]:
model = ResNet50(input_shape = IMAGE_Size,weights="imagenet",include_top = False)

94773248/94765736 [==============================] - 121s 1us/step


In [25]:
for layer in model.layers:
    layer.trainable = False

In [29]:
folders = glob("training/*")
folders

['training\\Parasitized', 'training\\Uninfected']

In [30]:
x=Flatten()(model.output)
predict = Dense(len(folders),activation='softmax')(x)

In [32]:
model=Model(inputs=model.input,outputs = predict)

In [36]:
opt = SGD(lr=INIT_LR,momentum=0.9)
model.compile(loss="binary_crossentropy",optimizer = opt,metrics = ["accuracy"])

In [37]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 64, 64, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 70, 70, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 32, 32, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 32, 32, 64)   256         conv1_conv[0][0]                 
______________________________________________________________________________________________

In [38]:
callbacks = [LearningRateScheduler(poly_decay)]
H = model.fit(
    x=trainGen,
    steps_per_epoch=totalTrain // BS,
    validation_data=valGen,
    validation_steps=totalVal // BS,
    epochs=NUM_EPOCHS,
    callbacks=callbacks,verbose=2)

Train for 602 steps, validate for 86 steps
602/602 - 602s - loss: 7.6581 - accuracy: 0.5012 - val_loss: 7.6728 - val_accuracy: 0.5011


In [39]:
testGen.reset()
predIdxs = model.predict(x=testGen, steps=(totalTest // BS) + 1)
predIdxs = np.argmax(predIdxs, axis=1)
print(classification_report(testGen.classes, predIdxs,
    target_names=testGen.class_indices.keys()))

              precision    recall  f1-score   support

 Parasitized       0.49      1.00      0.66      2726
  Uninfected       0.00      0.00      0.00      2786

    accuracy                           0.49      5512
   macro avg       0.25      0.50      0.33      5512
weighted avg       0.24      0.49      0.33      5512



C:\Users\Sujay J\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_accuracy")
plt.title("Training Loss and Accuracy on Dataset")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig(plott)